# The relationship between spatial coupling and incidence correlation in a 2-patch model
Ref: Keeling & Rohani, _Estimating spatial coupling in epidemiological systems: a mechanistic approach_, Ecol. Letters (2002) 5:20-29


In [ ]:
import numpy as np
import pandas as pd
from laser_core.propertyset import PropertySet
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

from laser_generic import Model
from laser_generic import Infection
from laser_generic import Susceptibility
from laser_generic import Transmission
from laser_generic import Births_ConstantPop
from laser_generic.importation import Infect_Random_Agents

from laser_generic.utils import set_initial_susceptibility_in_patch
from laser_generic.utils import seed_infections_in_patch

%load_ext line_profiler

f"{np.__version__=}"

In [ ]:
%%capture
import os

# Set parameters to match those of Keeling & Rohani, 2002.  They have a constant importation rate of 5.5e-5*sqrt(N) per day,
# which for this sim would be something like 40 per year.  I'm importing 6 per year, but I don't think that should be the big deal.
# more important is setting the birth, R0, and infectious period to their numbers and then seeing the
# connectivity-correlation relationship.
nticks = 36500
scenario = pd.DataFrame(data=[["patch1", 1e6], ["patch2", 1e6]], columns=["name", "population"])
parameters = PropertySet(
    {
        "seed": 4,
        "nticks": nticks,
        "verbose": True,
        "beta": 17 / 13,
        "inf_mean": 13,
        "cbr": 20.277294,
        "importation_period": 180,
        "importation_count": 3,
        "importation_start": 500,
    }
)

mu = (1 + parameters.cbr / 1000) ** (1 / 365) - 1
R0 = parameters.beta / (mu + 1 / parameters.inf_mean)
nsims = 100
i = 0
outputs = np.zeros((nsims, parameters.nticks, scenario.shape[0]))
# Create a folder to store the outputs
output_folder = "twopatchSIRoutputs"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for connection in np.logspace(-4, -1, nsims):
    model = Model(scenario, parameters)
    model.components = [
        Births_ConstantPop,
        Susceptibility,
        Transmission,
        Infection,
        Infect_Random_Agents,
    ]

    # Start them slightly asynchronously - different initial susceptibilities, infection only in 1 patch
    # Want to see how connectivity drives correlation over time.
    model.patches.network = np.array([[0, connection], [connection, 0]])
    set_initial_susceptibility_in_patch(model, 1, 1 / R0 + 0.02 * np.random.normal())
    set_initial_susceptibility_in_patch(model, 0, 1 / R0 + 0.02 * np.random.normal())
    seed_infections_in_patch(model, ipatch=0, ninfections=3)

    model.run()
    outputs[i, :, :] = model.patches.cases
    np.save(f"{output_folder}/twopatchSIRoutputs_{i}.npy", outputs[i, :, :])
    i += 1

In [ ]:
correlations = []
last_50_years = 50 * 365  # 50 years in days

for sim in range(outputs.shape[0]):
    patch1_data = outputs[sim, -last_50_years:, 0]
    patch2_data = outputs[sim, -last_50_years:, 1]
    correlation = np.corrcoef(patch1_data, patch2_data)[0, 1]
    correlations.append(correlation)

correlations = np.array(correlations)
print(correlations)

In [ ]:
from scipy.optimize import curve_fit

x = np.logspace(-4, -1, nsims)


# Define the function to fit
def func(x, psi, beta):
    return beta + x / (psi + x)


# Fit the curve
popt, pcov = curve_fit(func, x, correlations, p0=[0.01, 0], bounds=([1e-6, -0.00002], [1, 0.00002]))
psi_opt = popt[0]
beta_opt = popt[1]
plt.plot(x, correlations, ".")
plt.xscale("log")
# Plot the fitted curve
plt.plot(x, func(x, psi_opt, beta_opt), label=f"Fitted curve: y = {beta_opt:.4f} + x / ({psi_opt:.4f} + x)")
plt.legend()